# MaiChart AI - Colab Training

1. **Check GPU**: Runtime -> Change runtime type -> T4 GPU.

In [ ]:
!nvidia-smi

Wed Jan 21 18:03:59 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os

# 1. 确保回到根目录，彻底清理可能有问题的旧文件夹
%cd /content
!rm -rf /content/maimai-chat-ai

# 2. 重新克隆仓库
!git clone https://github.com/lvyirgil/maimai-chat-ai.git

# 3. 进入文件夹
%cd /content/maimai-chat-ai

# 4. 检查文件列表（现在应该能看到很多文件了，确认有 requirements.txt）
print("\n--- 当前目录下的文件 ---")
!ls

# 5. 如果看到了 requirements.txt，再安装依赖
if os.path.exists('requirements.txt'):
    !pip install -r requirements.txt
else:
    print("\n❌ 错误：仓库里依然没有找到 requirements.txt，请检查 GitHub 地址是否正确。")


/content
Cloning into 'maimai-chat-ai'...
remote: Enumerating objects: 1930, done.
remote: Counting objects: 100% (1930/1930), done.
remote: Compressing objects: 100% (1874/1874), done.
remote: Total 1930 (delta 72), reused 1900 (delta 50), pack-reused 0 (from 0)
Receiving objects: 100% (1930/1930), 4.40 MiB | 10.63 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/maimai-chat-ai

--- 当前目录下的文件 ---
check_cuda.py  GPU_TRAINING.md		 README.md	   tests
configs        IMPORT_TOOL_INSTALLED.md  requirements.txt  train_gpu.bat
data	       models			 scripts	   train_gpu.py
docs	       notebooks		 src		   train.ps1


In [8]:
import zipfile
import os
import shutil

zip_path = '/content/drive/MyDrive/maimai_data.zip'
# 目标：让 .txt 进 data/raw，让音频进 data/audio
target_base = '/content/maimai-chat-ai/data'

def force_unzip():
    if not os.path.exists(zip_path):
        print("❌ 错误：在网盘里没找到 maimai_data.zip，请确保 drive.mount 已运行且文件名正确！")
        return

    # 先清理一遍空文件夹
    shutil.rmtree(os.path.join(target_base, 'audio'), ignore_errors=True)
    shutil.rmtree(os.path.join(target_base, 'raw'), ignore_errors=True)
    os.makedirs(os.path.join(target_base, 'audio'), exist_ok=True)
    os.makedirs(os.path.join(target_base, 'raw'), exist_ok=True)

    print("正在强力解压中，请稍后（4.6GB 可能需要 2-3 分钟）...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        for info in z.infolist():
            # 1. 修复编码
            try:
                name = info.filename.encode('cp437').decode('gbk')
            except:
                name = info.filename

            name = name.replace('\\', '/')

            # 2. 识别文件类型并分类存放
            if name.endswith(('.mp3', '.wav', '.ogg', '.flac', '.MP3')):
                # 无论原来路径在哪，都只要文件名，强行塞进 data/audio/
                fname = os.path.basename(name)
                if fname:
                    with z.open(info) as source, open(os.path.join(target_base, 'audio', fname), 'wb') as target:
                        shutil.copyfileobj(source, target)

            elif name.endswith('.txt'):
                # 强行塞进 data/raw/
                fname = os.path.basename(name)
                if fname:
                    with z.open(info) as source, open(os.path.join(target_base, 'raw', fname), 'wb') as target:
                        shutil.copyfileobj(source, target)

    print(f"✅ 解压完成！")
    print(f"当前音频数: {len(os.listdir(os.path.join(target_base, 'audio')))}")
    print(f"当前谱面数: {len(os.listdir(os.path.join(target_base, 'raw')))}")

force_unzip()

正在强力解压中，请稍后（4.6GB 可能需要 2-3 分钟）...
✅ 解压完成！
当前音频数: 1757
当前谱面数: 1810


In [9]:
# 1. 进入项目文件夹
%cd /content/maimai-chat-ai

# 2. 确认一下文件在不在（应该能看到 requirements.txt）
!ls

# 3. 再安装依赖
!pip install -r requirements.txt

/content/maimai-chat-ai
check_cuda.py  GPU_TRAINING.md		 README.md	   tests
configs        IMPORT_TOOL_INSTALLED.md  requirements.txt  train_gpu.bat
data	       models			 scripts	   train_gpu.py
docs	       notebooks		 src		   train.ps1


In [ ]:
!python -m src.data.preprocess

警告: 找不到 てらてら 的音频文件
警告: 找不到 お呪い 的音频文件
警告: 找不到 Pixel Galaxy 的音频文件
警告: 找不到 Eureka 的音频文件
警告: 找不到 超最終鬼畜妹フランドール・S 的音频文件
警告: 找不到 アンダーキッズ 的音频文件
警告: 找不到 つづみぐさ 的音频文件
警告: 找不到 ㋰責任集合体 的音频文件
警告: 找不到 美夜月鏡 的音频文件
警告: 找不到 シスターシスター 的音频文件
警告: 找不到 Make Up Your World feat. キョンシーのCiちゃん & らっぷびと 的音频文件
警告: 找不到 sølips 的音频文件
警告: 找不到 サウンドプレイヤー 的音频文件
警告: 找不到 HYP3RTRIBE 的音频文件
警告: 找不到 デビルじゃないもん 的音频文件
警告: 找不到 しゅ～しん？変身☆ハカイシンzzZ 的音频文件
警告: 找不到 FLΛME-FRΦST 的音频文件
警告: 找不到 Flashback 的音频文件
警告: 找不到 最っ高のエンタメだ!! 的音频文件
警告: 找不到 Beginning together! 的音频文件
警告: 找不到 Ref_rain (for 7th Heaven) 的音频文件
警告: 找不到 Geranium 的音频文件
警告: 找不到 命テステス 的音频文件
警告: 找不到 MYTH Re：LEASE 的音频文件
警告: 找不到 [覚] Hainuwele 的音频文件
警告: 找不到 テトリス 的音频文件
警告: 找不到 Shining Ray ～僕らの絆～ 的音频文件
警告: 找不到 あいたい星人 的音频文件
警告: 找不到 Colorfull_Encounter 的音频文件
警告: 找不到 Hainuwele 的音频文件
警告: 找不到 偉大なる悪魔は実は大天使パトラちゃん様なのだ！ 的音频文件
警告: 找不到 抜錨 的音频文件
警告: 找不到 VOLT 的音频文件
警告: 找不到 Cider P@rty 的音频文件
警告: 找不到 ハオ 的音频文件
警告: 找不到 はいよろこんで 的音频文件
警告: 找不到 無間嫉妬劇場『666』 的音频文件
警告: 找不到 NO ONE YES MAN 的音频文件
警告: 找不到 [嘘] ライアーダン

In [ ]:
import os

# 1. 定义网盘中的备份路径
drive_models_path = '/content/drive/MyDrive/maimai_models_sync'
local_models_path = '/content/maimai-chat-ai/models'

# 2. 创建网盘目录
!mkdir -p {drive_models_path}

# 3. 如果本地 models 文件夹已存在且不是软链接，先转移文件并替换为软链接
if os.path.exists(local_models_path) and not os.path.islink(local_models_path):
    print("正在建立实时同步软链接...")
    # 移动现有模型到网盘
    !cp -rn {local_models_path}/* {drive_models_path}/ 2>/dev/null
    # 删除本地文件夹并创建指向网盘的链接
    !rm -rf {local_models_path}
    !ln -s {drive_models_path} {local_models_path}
    print("✅ 实时同步已就绪：现在模型保存会直接写入网盘。")
elif not os.path.exists(local_models_path):
    !ln -s {drive_models_path} {local_models_path}
    print("✅ 已创建同步软链接。")

# 4. 启动训练（无需额外的备份线程）
!python train_gpu.py --config configs/default.yaml --resume {local_models_path}/best.pt

正在建立实时同步软链接...
✅ 实时同步已就绪：现在模型保存会直接写入网盘。
载入配置文件: configs/default.yaml

GPU 信息
✓ CUDA 可用
✓ GPU 数量: 1

GPU 0: Tesla T4
  - 计算能力: 7.5
  - 总显存: 14.7 GB
  - 当前显存占用: 0.0 GB

✓ PyTorch 版本: 2.9.0+cu126
✓ 混合精度: 启用 (FP16)

训练配置:
  - 设备: cuda
  - Batch Size: 4
  - 学习率: 1e-4
  - 混合精度: 启用
  - 最大轮数: 100
  - 梯度累积步数: 4

✓ 检测到 GPU，启用混合精度和优化参数
加载样本: 100% 1757/1757 [00:52<00:00, 33.59it/s]
加载了 1757 个样本
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
从 /content/maimai-chat-ai/models/best.pt 加载检查点 (epoch 5)
开始训练，设备: cuda
训练集大小: 1405
验证集大小: 175
Epoch 0:   0% 0/351 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 0: 352it [09:17,  1.58s/it, loss=0.2331, lr=1.00e-04]
Epoch 0: 

In [3]:
!git fetch origin && git reset --hard origin/main

fatal: not a git repository (or any of the parent directories): .git


In [ ]:
# 1. 将预处理后的 .npz 文件打包（建议直接在 Colab 运行）
!zip -r processed_data.zip /content/maimai-chat-ai/data/processed

# 2. 复制到网盘根目录
!cp processed_data.zip /content/drive/MyDrive/
print("✅ 预处理数据已成功备份到网盘！")

In [ ]:
# 直接将预处理好的数据还原到项目目录
!mkdir -p /content/maimai-chat-ai/data/processed
!unzip -q /content/drive/MyDrive/processed_data.zip -d /content/maimai-chat-ai/data/

# 检查文件是否到位
!ls /content/maimai-chat-ai/data/processed | head -n 5